In [ ]:
import pandas as pd
import sklearn
import scipy
import matplotlib.pyplot as plt

In [ ]:
import os

In [ ]:
data_location = os.path.normpath('../data')

In [ ]:
file = os.path.join(data_location, "hackathon_low_mixed_venous_oximetry.csv")
# file = os.path.join(data_location, "hackathon_low_cardiac_output.csv")

In [ ]:
df = (
    pd.read_csv(file)
)
feature_columns = [col for col in df.columns if col not in ["event_count",
                                                            "ClassificationLabel",
                                                           "subject_id"]  and "Regression" not in col]

if 'cardiac_output' in file:
    regression_label = "RegressionLabel-CardiacIndex"
else:
    regression_label = "RegressionLabel-SvO2"

# investigations

In [ ]:
df.groupby("subject_id").count().event_count.value_counts()
df.isna().sum(axis=1).value_counts()

# Preprocessing

In [ ]:
df = (
    df
    .assign(ClassificationLabel = df.ClassificationLabel=="Positive")
    .assign(gender=df.gender=="F")
#     .loc[lambda df: ~df.gender]
)


means = df[feature_columns+["subject_id"]].groupby("subject_id").mean()

to_drop = []
for row_id, row in df.iterrows():
    for col, elem in row.items():
        if pd.isna(elem):
            new = means.loc[row.subject_id, col]
            if pd.isna(new):
                to_drop.append(row_id)
            else:    
                df.loc[row_id, col] = new

df = df.loc[lambda df: ~df.index.isin(to_drop)]
                
# for col in feature_columns:
#     df[col].fillna(df[col].mean(), inplace=True)

In [ ]:
import sklearn.covariance
env = sklearn.covariance.EllipticEnvelope()

env.fit(df[feature_columns].values)

outliers = env.predict(df[feature_columns].values)
print(f"Outliers count= {sum(outliers==-1)}")
df = df.loc[outliers==1]


len(df)

df = df.reset_index(drop=True)

In [ ]:
%matplotlib inline
for col in feature_columns:
    print(col)
    if col != 'gender':
        plt.scatter(df[col], df[regression_label])
        plt.show()
        
        

In [ ]:
%matplotlib inline
for col in feature_columns:
    print(col)
    if col != 'gender':
#         df[col].hist(bins=20)
#         plt.show()
        pass

df["bmi"] = (df.weight/(df.height*df.height))

df.bmi.hist(bins=20)

plt.show()
df.bmi.median()

# df = df.loc[lambda df: df.age_years>69]

# Train test split

In [ ]:
subjects = list(set(df.subject_id))
N = int(len(subjects)/3*2)
train_subjects = subjects[:N]
test_subjects = subjects[N:]



df_train=df.loc[df.subject_id.isin(train_subjects)]
df_test=df.loc[df.subject_id.isin(test_subjects)]

print(len(df_test))
print(len(df_train))

# Classification

In [ ]:

# fit xgboost on an imbalanced classification dataset
from numpy import mean
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from xgboost.sklearn import XGBClassifier

In [ ]:
df_train[regression_label].hist(bins=30)

In [ ]:
from sklearn.model_selection import train_test_split
X = df_train[feature_columns].values
y = df_train[regression_label]<60

X_test = df_test[feature_columns].values
y_test = df_test[regression_label]<60

In [ ]:
from sklearn import tree
import sklearn.ensemble

clf = tree.DecisionTreeClassifier(max_depth=2)
# clf = sklearn.ensemble.GradientBoostingClassifier()
clf = sklearn.ensemble.RandomForestClassifier()


# clf = XGBClassifier(scale_pos_weight=1, max_depth=3, subsample=1, colsample_bytree=1, min_child_weight=40)



In [ ]:
# evaluate model
scores = cross_val_score(clf, X, y, scoring='roc_auc', n_jobs=-1)
# summarize performance
print('Mean ROC AUC: %.5f' % mean(scores))

In [ ]:
clf = clf.fit(X,y)

In [ ]:
# fig = plt.figure(figsize=(25,20))
# _ = tree.plot_tree(clf)

In [ ]:
y_test_predict = clf.predict(X_test)
import sklearn.metrics as metrics
conf_matrix = metrics.confusion_matrix(y_test, y_test_predict)
pd.DataFrame(conf_matrix,
            index=[f"label_{i}" for i in range(0, max(y_test)+1)],
            columns=[f"pred_{i}" for i in range(0, max(y_test)+1)])

In [ ]:
from yellowbrick.classifier import ClassBalance, ROCAUC, ClassificationReport, ClassPredictionError

In [ ]:

clf.target_type_ = int
rocauc = ROCAUC(clf, size=(1080, 720), classes=[0,1])

rocauc.score(X_test, y_test)  
r = rocauc.poof()

# Regression

In [ ]:
%matplotlib notebook

In [ ]:
from sklearn.model_selection import train_test_split

ff = feature_columns.copy()
ff.remove('gender')
X = df_train[ff].values
y = df_train[regression_label]

X_test = df_test[ff].values
y_test = df_test[regression_label]

In [ ]:
clf = sklearn.linear_model.Lasso(alpha=0.1, fit_intercept=True,
                                 normalize=True, precompute=False, copy_X=True,
                                 max_iter=1000, tol=0.0001, warm_start=False,
                                 positive=False, random_state=True, selection='cyclic')

clf = sklearn.linear_model.LinearRegression()
from sklearn.ensemble import GradientBoostingRegressor
clf = GradientBoostingRegressor(max_depth=1, )


# clf = sklearn.svm.SVR()

In [ ]:
clf.fit(X,y)
y_pred = clf.predict(X_test)

sklearn.metrics.mean_squared_error(y_test, y_pred)

In [ ]:
%matplotlib inline

In [ ]:
plt.scatter(y_test, y_pred)

# Dimensionality Reduction

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
import sklearn.manifold

In [ ]:
ff = feature_columns.copy()
ff.remove('gender')

X_for_pca = df[ff].values

In [ ]:
pca = PCA(n_components=3)
# pca = sklearn.manifold.TSNE(n_components=3)

scaler = MinMaxScaler()
scaler.fit(X_for_pca)
X_normalized = scaler.transform(X_for_pca)

pca.fit(X_normalized)

In [ ]:
# import numpy as np
# np.argmax(pca.components_[0])

In [ ]:
X_pca = pca.fit_transform(X_normalized[df_test.index])

In [ ]:
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

In [ ]:
xx = X_pca[:,1]
yy = X_pca[:,2]

cm = plt.cm.get_cmap('RdYlBu')
top = plt.cm.get_cmap('Oranges_r', 60)
bottom = plt.cm.get_cmap('Blues', 100)

newcolors = np.vstack((top(np.linspace(0, 1, 60)),
                       bottom(np.linspace(0, 1, 100))))

newcmp = ListedColormap(newcolors, name='OrangeBlue')
fig, ax = plt.subplots()
aa = ax.scatter(xx,yy,c=y_test, alpha=1, vmin=0, vmax=y.max(), s=35, cmap=newcmp)
plt.colorbar(aa)

In [ ]:
%matplotlib notebook

In [ ]:
nn =1800
xx = X_pca[:nn,0]
yy = X_pca[:nn,1]
zz = X_pca[:nn, 2]
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter(xx,yy,zz,c=y_test, alpha=1, vmin=0, vmax=y.max(), cmap=newcmp, marker = 'o')
# plt.colorbar(aa)